In [1]:
from pandora_bayesopt.test_functions.lunar_lander import LunarLanderProblem

In [2]:
import torch
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.acquisition.budgeted_multi_step_ei import BudgetedMultiStepLookaheadEI
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer

In [3]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
seed = 42
torch.manual_seed(seed)

In [5]:
dim = 12
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [6]:
def objective_cost_function(X):
    return LunarLanderProblem()(X)

In [7]:
init_x

tensor([[9.9751e-01, 1.0437e-01, 8.2298e-01, 4.1943e-01, 5.2832e-01, 5.6241e-02,
         4.5529e-01, 7.4920e-01, 2.2107e-01, 6.1770e-01, 8.1009e-01, 2.1229e-01],
        [3.8436e-02, 8.8182e-01, 6.5063e-01, 6.6659e-01, 8.2093e-01, 5.9658e-01,
         2.3631e-01, 8.0697e-01, 1.6714e-01, 7.6784e-01, 6.0348e-01, 9.3257e-01],
        [5.3768e-01, 3.9881e-01, 3.6134e-01, 2.8972e-01, 6.5357e-01, 2.3477e-01,
         3.7597e-01, 3.2264e-01, 9.9217e-01, 3.4845e-01, 1.7679e-02, 3.1838e-01],
        [4.2821e-01, 3.8280e-01, 1.6977e-01, 8.6547e-02, 4.0842e-01, 3.5837e-01,
         4.7025e-01, 4.8702e-01, 2.5286e-01, 8.2489e-01, 6.1748e-01, 3.7138e-01],
        [4.6918e-01, 2.1094e-01, 7.5127e-01, 8.6537e-02, 5.5801e-01, 2.7181e-01,
         6.3428e-01, 7.5547e-01, 9.0873e-01, 4.0887e-02, 9.4764e-02, 1.7455e-01],
        [4.4954e-01, 5.1475e-02, 2.2210e-01, 2.4930e-01, 4.2032e-01, 5.2357e-01,
         9.1859e-02, 3.5245e-01, 3.5383e-01, 4.5246e-01, 5.5045e-01, 3.5966e-01],
        [3.8447e-01, 3

In [8]:
objective_X, cost_X = objective_cost_function(init_x)
objective_X, cost_X

571 232.21358713670193
932 144.6508599521895


(tensor([ -54.2268,  -83.5435,  -50.9528, -107.6307,    8.6476, -114.2625,
          -93.6529, -130.8711, -161.5588,  -65.1199,  -65.8553, -126.5917,
         -161.8550, -104.2003, -123.6243,  -69.5658, -161.1728, -116.6186,
         -252.5206,  -60.6764, -104.8332, -127.6867,  -80.2447, -131.7593,
         -126.4442, -129.8680]),
 tensor([ 4.8825,  5.5512,  5.1900,  4.5575, 13.2950,  4.5237,  5.0663,  4.4875,
          7.6712,  4.6775,  5.1175,  4.4238,  6.3113,  4.4837,  4.4675,  5.0687,
          6.4562,  4.4238,  9.0488,  4.7825,  5.5462,  6.2012,  4.6762,  4.4662,
          4.5200,  4.4337]))

In [7]:
Optimizer = BayesianOptimizer(
        dim=dim, 
        maximize=True, 
        initial_points=init_x,
        objective_cost=objective_cost_function, 
        input_standardize=True
    )
Optimizer.run_until_budget(
    budget = 10,
    acquisition_function_class=BudgetedMultiStepLookaheadEI
)

571 232.21358713670193
932 144.6508599521895
154.32999999999998


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)